In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('train_combined.csv')
test = pd.read_csv('test_combined.csv')

In [ ]:
train = train.drop(columns=['id_titik_mulai','id_titik_akhir','titik1','titik2','lat1','long1','lat2','long2'])
test = test.drop(columns=['id_titik_mulai','id_titik_akhir','titik1','titik2','lat1','long1','lat2','long2'])

In [ ]:
test.shape

(127489, 6)

In [ ]:
train.shape

(398648, 6)

#Preprocessing

In [ ]:
train['waktu_setempat'] = pd.to_datetime(train['waktu_setempat'])
train['tanggal'] = train['waktu_setempat'].dt.day
train['jam'] = train['waktu_setempat'].dt.strftime('%H:%M:%S')
train['jam'] = train['waktu_setempat'].dt.hour + train['waktu_setempat'].dt.minute / 60
train.drop(columns=['waktu_setempat'], inplace=True)

test['waktu_setempat'] = pd.to_datetime(test['waktu_setempat'])
test['tanggal'] = test['waktu_setempat'].dt.day
test['jam'] = test['waktu_setempat'].dt.strftime('%H:%M:%S')
test['jam'] = test['waktu_setempat'].dt.hour + test['waktu_setempat'].dt.minute / 60
test.drop(columns=['waktu_setempat'], inplace=True)

#Encoder

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
id_jalan_kelompok_train = train[['id_jalan', 'kelompok']]
encoded_train = encoder.fit_transform(id_jalan_kelompok_train)

id_jalan_kelompok_test =test[['id_jalan', 'kelompok']]
encoded_test = encoder.transform(id_jalan_kelompok_test)

In [ ]:
# Ubah hasil encoding menjadi DataFrame
encoded_train_df = pd.DataFrame(encoded_train, columns=['encoded_id_jalan', 'encoded_kelompok'])
encoded_test_df = pd.DataFrame(encoded_test, columns=['encoded_id_jalan', 'encoded_kelompok'])

# Gabungkan hasil encoding dengan data train dan test
train_clean = pd.concat([train, encoded_train_df], axis=1)
test_clean = pd.concat([test, encoded_test_df], axis=1)


#Model

In [ ]:
train_clean.head()

,id_jalan,rerata_kecepatan,kelompok,maxspeed_mph,jarak_km,tanggal,jam,encoded_id_jalan,encoded_kelompok
0,691007296,29.126,1425033102_21390008,30,0.007931,1,1.0,19.0,111.0
1,691007296,28.999,1425033102_21390008,30,0.007931,1,1.0,19.0,111.0
2,691007296,34.772,1425033102_21390008,30,0.007931,1,2.0,19.0,111.0
3,691007296,32.222,1425033102_21390008,30,0.007931,1,2.0,19.0,111.0
4,691007296,37.338,1425033102_21390008,30,0.007931,1,3.0,19.0,111.0


In [ ]:
X_train = train_clean.drop(['id_jalan', 'rerata_kecepatan', 'kelompok'], axis= 1)
X_test = test_clean.drop(['id_jalan', 'kelompok', 'id'], axis= 1)
y_train = train_clean['rerata_kecepatan']

In [ ]:
X_train.head()

,maxspeed_mph,jarak_km,tanggal,jam,encoded_id_jalan,encoded_kelompok
0,30,0.007931,1,1.0,19.0,111.0
1,30,0.007931,1,1.0,19.0,111.0
2,30,0.007931,1,2.0,19.0,111.0
3,30,0.007931,1,2.0,19.0,111.0
4,30,0.007931,1,3.0,19.0,111.0


In [ ]:
X_test.head()

,maxspeed_mph,jarak_km,tanggal,jam,encoded_id_jalan,encoded_kelompok
0,30,0.044870,23,0.0,3.0,387.0
1,30,0.035285,23,0.0,18.0,106.0
2,20,0.030340,23,0.0,8.0,121.0
3,30,0.017702,23,0.0,18.0,326.0
4,30,0.117762,23,0.0,14.0,293.0


#Model

In [ ]:
import xgboost as xgb

In [ ]:
SEED = 42
model = xgb.XGBRegressor(n_jobs=-1, random_state=SEED)
model.fit(X_train, y_train)

preds = model.predict(X_test)

In [ ]:
output = pd.DataFrame({'id': test_clean['id'],
                       'rerata_kecepatan': preds})

In [ ]:
output.head()

,id,rerata_kecepatan
0,0,43.197540
1,1,39.695927
2,2,36.400642
3,3,39.960766
4,4,32.019432


In [ ]:
output.describe()

,id,rerata_kecepatan
count,127489.000000,127489.000000
mean,63744.000000,36.099548
std,36803.048572,6.627234
min,0.000000,13.081491
25%,31872.000000,32.225433
50%,63744.000000,36.513382
75%,95616.000000,40.768421
max,127488.000000,52.481167


In [ ]:
output.shape

(127489, 2)

In [ ]:
test.shape

(127489, 7)

In [ ]:
output.to_csv('submit2.csv', index=False)